<a href="https://colab.research.google.com/github/ahnjihyun/BOOK_Deep-learning-from-scratch-2/blob/main/1-3%EC%8B%A0%EA%B2%BD%EB%A7%9D%EC%9D%98_%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.3 신경망의 학습

##### (p54~) 1.3.4 계산 그래프
역전파 메서드 추가

In [1]:
class MatMul: # matrix multiply 
  def __init__(self,W):
    self.params = [W] # 학습하는 매개변수를 params에 보관
    self.grads = [np.zeros_like(W)] # 거기에 대응시키는 형태로 기울기를 grads에 보관
    self.x = None

  def forward(self,x):
    W, = self.params
    out = np.matmul(x,W)
    self.x = x
    return out

  def backward(self, dout):
    W, = self.params
    dx = np.matmul(dout, W.T)
    dW = np.matmul(self.x.T, dout)
    self.grads[0][...] = dW # 기울기값을 설정하는 코드 
    # 생략기호(ellipsis, '...') 사용 => 넘파이 배열이 가리키는 메모리 위치를 고정시킨 다음, 그 위치에 원소들을 덮어씀(깊은 복사)
    return dx

##### 1.3.5 기울기 도출과 역전파 구현 
Sigmoid 계층

In [2]:
class Sigmoid:
  def __init__(self):
    self.params, self.grads = [], []
    self.out = None

  def forward(self,x):
    out = 1 / (1+np.exp(-x))
    self.out = out
    return out

  def backward(self, dout):
    dx = dout * (1.0 - self.out) * self.out
    return dx

Affine 계층

In [3]:
class Affine:
  def __init__(self,W,b):
    self.parans = [W,b]
    self.grads = [np.zeros_like(W), np.zeros_like(b)]
    self.x = None 

  def forward(self,x):
    W,b = self.params
    out = np.matmul(x,W) + b
    self.x = x
    return out

  def backward(self, dout):
    W,b = self.params
    dx = np.matmul(dout, W.T)
    dW = np.matmul(self.x.T, dout)
    db = np.sum(dout, axis=0)

    self.grads[0][...] = dW
    self.grads[1][...] = db
    return dx

##### (p60) 1.3.6 가중치 갱신

신경망의 학습은 다음 순서로 수행합니다

> - **1단계: 미니배치**
> 
>     훈련데이터 중에서 무작위로 다수의 데이터를 골라낸다 (배치=덩어리)
> 
> - **2단계: 기울기 계산**
> 
>     오차역전파법으로 각 가중치 매개변수에 대한 손실 함수의 기울기를 구한다
> 
> - **3단계: 매개변수 갱신**
> 
>     기울기를 사용하여 가중치 매개변수를 갱신한다
> 
> - **4단계: 반복**
> 
>     1~3단계를 필요한 만큼 반복한다

오차역전파법으로 가중치의 기울기를 얻는다.

이 기울기는 현재의 가중치 매개변수에서 손실을 가장 크게 하는 방향을 가리킨다.

따라서 매개변수를 그 기울기와 반대 방향으로 갱신하면 손실을 줄일 수 있다

=> 경사하강법(Gradient Discent)


##### (p61) 확률적 경사하강법(Stochastic Gradient Discent) 구현

In [4]:
class SGD:
  def __init__(self, lr=0.01): # 초기화 인수 lr은 학습률(learning rate)를 의미
    self.lr = lr

  def update(self, params, grads):
    for i in range(len(params)):
      params[i] -= self.lr * grads[i]